# 3D Reconstruction

## Directory
Download the data from NeuroGT (https://ssbd.riken.jp/neurogt/) and create the following directory before execution.

```
.
└── 3D-R/
    ├── 3D-R.ipynb
    └── dataset/
        ├── 1
        ├── 2
        ├── 3
        ├── ...
        └── 84/
            └── G2AN_woTM_L/
                └── G2AN_woTM_xxxx_Lxxx_x.png
```

In [ ]:
import os
import urllib.request

#Creating Directories
def mkdir(path):
    if not os.path.exists(path):#ディレクトリがなかったら
        os.mkdir(path)#作成したいフォルダ名を作成
        
        
# 1 if all imaging datasets are download from the internet,
# 0 if some imaging datasets have already been download locally, 
#    for example: 
#        dataset/
#           1/
#              G2AN_TM11.5_G/
#.          2/
#              G2AN_TM11.5_L/
DOWNLOAD_ALL = 0

if DOWNLOAD_ALL == 0:
    txt = 'name.txt'
    with open(txt) as f:
        l_strip = [s.rstrip() for s in f.readlines()]
        #print(len(l_strip), l_strip)
    
    n=0
    for i in range(67,69):
        dataset = 'dataset/'
        mkdir(dataset)
        
        new = dataset+str(i)+'/'
        mkdir(new)
        
        if i%2 == 0:
            url='https://ssbd.riken.jp/neurogt/zip/'+l_strip[i//2-1]+str('L')+'.zip'
            save_name='dataset/'+str(i)+'/'+l_strip[i//2-1]+str('L')+'/'+l_strip[i//2-1]+str('L')+'.zip'
            n=n+1
        
        else:
            url='https://ssbd.riken.jp/neurogt/zip/'+l_strip[i//2]+str('G')+'.zip'
            save_name='dataset/'+str(i)+'/'+l_strip[i//2]+str('G')+'/'+l_strip[i//2]+str('G')+'.zip'
    
        urllib.request.urlretrieve(url, save_name)
        print(url, save_name)
        
            
elif DOWNLOAD_ALL == 1:
    txt = 'name.txt'
    with open(txt) as f:
        l_strip = [s.rstrip() for s in f.readlines()]
        print(l_strip)
    
    n=0
    for i in range(1,85):
        dataset = 'dataset/'
        mkdir(dataset)
        
        new = dataset+str(i)+'/'
        mkdir(new)
        
        """
        if i%2 == 0:
            url='https://ssbd.riken.jp/neurogt/zip/'+l_strip[n]+str('L')+'.zip'
            save_name='dataset/'+str(i)+'/'+l_strip[n]+str('L')+'/'+l_strip[n]+str('L')+'.zip'
            n=n+1
        
        else:
            url='https://ssbd.riken.jp/neurogt/zip/'+l_strip[n]+str('G')+'.zip'
            save_name='dataset/'+str(i)+'/'+l_strip[n]+str('G')+'/'+l_strip[n]+str('G')+'.zip'
        """
        
        if i%2 == 0:
            url='https://ssbd.riken.jp/neurogt/zip/'+l_strip[i//2-1]+str('L')+'.zip'
            save_name='dataset/'+str(i)+'/'+l_strip[i//2-1]+str('L')+'/'+l_strip[i//2-1]+str('L')+'.zip'
            
        
        else:
            url='https://ssbd.riken.jp/neurogt/zip/'+l_strip[i//2]+str('G')+'.zip'
            save_name='dataset/'+str(i)+'/'+l_strip[i//2]+str('G')+'/'+l_strip[i//2]+str('G')+'.zip'
            
        urllib.request.urlretrieve(url, save_name)
        print(url, save_name)

## Save the contour of the object as a 3D point cloud

In [1]:
import glob
import matplotlib.pyplot as plt
import cv2
import re
import os
import numpy as np
import pandas as pd
import csv
from skimage.io import imread, imshow
from skimage import data
from skimage.util import img_as_ubyte
from skimage.filters.rank import entropy
from skimage.morphology import disk
from skimage.color import rgb2hsv, rgb2gray, rgb2yuv
import concurrent.futures
import logging
import time
from tqdm.notebook import trange
import warnings

In [2]:
#Creating Directories
def mkdir(path):
    if not os.path.exists(path):#ディレクトリがなかったら
        os.mkdir(path)#作成したいフォルダ名を作成
    
#Sorting Numbers
def numericalSort(value):
    numbers = re.compile(r'(\d+)')
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

def kmeans(data_in):
    data = np.floor(data_in*255)
    # ラベルの初期化
    labels = np.random.randint(0,2,data.shape[0])

    # 終了条件
    OPTIMIZE_EPSILON = 1

    m_0_old = -np.inf
    m_1_old = np.inf

    for i in range(1000):
    # それぞれの平均の計算
        m_0 = data[labels==0].mean()
        m_1 = data[labels==1].mean()
    # ラベルの再計算
        labels[np.abs(data-m_0) < np.abs(data-m_1)] = 0
        labels[np.abs(data-m_0) >= np.abs(data-m_1)] = 1
    #     終了条件
        if np.abs(m_0 - m_0_old) + np.abs(m_1 - m_1_old) < OPTIMIZE_EPSILON:
            break
        m_0_old = m_0
        m_1_old = m_1
        
    # 初期値によって，クラスが変化するため上界の小さい方を採用
    thresh_kmeans = np.minimum(data[labels==0].max(),data[labels==1].max())
    
    return thresh_kmeans/255


def compare(com_path, files, n):
    print(com_path, files, n)
    img_in = imread(files)
    dst = cv2.GaussianBlur(img_in, (5, 5), sigmaX=1)
    img_in = rgb2gray(img_in)

    dst = rgb2gray(dst)

    neiborhood = np.ones((5,5),np.uint8)
    
    entropy_image_1 = entropy(img_in, neiborhood)
    entropy_image_2 = entropy(dst, neiborhood)
    
    scaled_entropy_1 = entropy_image_1 / entropy_image_1.max() # ガウシアンなし
    scaled_entropy_2 = entropy_image_2 / entropy_image_2.max() # ガウシアンあり

    a1 = np.floor(scaled_entropy_1*255)
    a2 = np.floor(scaled_entropy_2*255)
    
    a1 = a1.astype('uint8')
    a2 = a2.astype('uint8')
    
    # 画像をヒストグラム化する
    image1_hist = cv2.calcHist([a1], [0], None, [256], [0, 256])
    image2_hist = cv2.calcHist([a2], [0], None, [256], [0, 256])
    
    #print(cv2.compareHist(image1_hist, image2_hist, 0))
    
    np.savetxt(com_path+str(n)+'_compare.txt', [cv2.compareHist(image1_hist, image2_hist, 0)], fmt ='%.6f')

#Thresholding by k-means after entropy filter
def threshhold(th_path, img_files, com_files, k_num, n):
    pin = 0
    # データの読み込み
    img = cv2.imread(img_files)
    #ガウシアンフィルタ
    #img = cv2.GaussianBlur(img, (5, 5), sigmaX=1)
    # グレースケール変換
    shawl_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    neiborhood = np.ones((5,5),np.uint8)
    
    scaled_entropy = shawl_gray / shawl_gray.max()
    entropy_image = entropy(scaled_entropy, neiborhood)
    scaled_entropy = entropy_image / entropy_image.max()
    
    a_ = np.floor(scaled_entropy*255)
    a = a_.astype(np.int64)
    data = a.reshape(-1)
    
    # k-means
    # ラベルの初期化
    labels = np.random.randint(0,2,data.shape[0])

    # 終了条件
    OPTIMIZE_EPSILON = 1

    m_0_old = -np.inf
    m_1_old = np.inf

    for i in range(1000):
    # それぞれの平均の計算
        m_0 = data[labels==0].mean()
        m_1 = data[labels==1].mean()
    # ラベルの再計算
        labels[np.abs(data-m_0) < np.abs(data-m_1)] = 0
        labels[np.abs(data-m_0) >= np.abs(data-m_1)] = 1
    #     終了条件
        if np.abs(m_0 - m_0_old) + np.abs(m_1 - m_1_old) < OPTIMIZE_EPSILON:
            break
        m_0_old = m_0
        m_1_old = m_1

    # 初期値によって，クラスが変化するため上界の小さい方を採用
    thresh_kmeans = np.minimum(data[labels==0].max(),data[labels==1].max())
    
    thresh_kmeans = thresh_kmeans/255
    
    mask = scaled_entropy > thresh_kmeans
    
    c = mask*255
    
    #形式変換
    c = c.astype('uint8')
    
    neiborhood = np.ones((3,3),np.uint8)
    
    print(img_files.split('/')[-1].split('.png')[0])
    
    if com_files > k_num:
        ##ピンボケの場合
        #収縮
        img_erode = cv2.erode(c,neiborhood,iterations=10)
        #膨張
        img_dilate = cv2.dilate(img_erode,neiborhood,iterations=10)
        cv2.imwrite(th_path+img_files.split('/')[-1].split('.png')[0]+'_k-means.png', img_dilate)

    else:
        ##気泡の場合
        #膨張
        img_dilate = cv2.dilate(c,neiborhood,iterations=4)
        #収縮
        img_erode = cv2.erode(img_dilate,neiborhood,iterations=4)
        cv2.imwrite(th_path+img_files.split('/')[-1].split('.png')[0]+'_k-means.png', img_erode)
    
    
#get graphics size 
def getSize(files,path):
    x=[]
    y=[]
    for n in range(len(files)):
        img = cv2.imread(files[n])
        h, w = img.shape[:2]
        x.append(w)
        y.append(h)
    
    xy = [max(x),max(y)]
    np.savetxt(path+'size.txt', xy, fmt ='%.0f')
    
    #print("getSize comp")

    return x, y


def padding(pad_path,x,y,files,n):
    a_y, a_x = y[n], x[n]
    
    A_x = (max(x) - a_x)//2
    
    A_y = (max(y) - a_y)//2
    
    img_original = cv2.imread(files[n])
    
    img_pad = cv2.copyMakeBorder(img_original, A_y, A_y, A_x, A_x, cv2.BORDER_CONSTANT, value=(0,0,0))

    cv2.imwrite(pad_path+files.split('/')[-1].split('.png')[0]+'_padding.png', img_pad)
    
    #print("padding comp")
    
#並列処理用関数
def padding_parallel(pad_path,xy,files,k_files,n):
    
    a_y, a_x = int(xy[1]), int(xy[0])
    #print(a_y, a_x)
    img_original = cv2.imread(k_files)
    #print(files[n])
    h, w = img_original.shape[:2]
    
    A_x = (a_x - w)//2
    A_y = (a_y - h)//2
    
    img_pad = cv2.copyMakeBorder(img_original, A_y, A_y, A_x, A_x, cv2.BORDER_CONSTANT, value=(0,0,0))
    
    #print(img_pad)
    
    cv2.imwrite(pad_path+files.split('/')[-1].split('.png')[0]+'_padding.png', img_pad)
    
    #print("padding comp")
    

def contour(pad_path,tar_path):

    # 入力画像の読み込み
    files=sorted(glob.glob(pad_path+'*.png'), key=numericalSort)

    for n in range(len(files)):

        img_original = cv2.imread(files[n])
        
        #画像の面積
        h, w = img_original.shape[:2]
        area = h*w
        
        # グレースケール変換
        gray = cv2.cvtColor(img_original, cv2.COLOR_RGB2GRAY)

        contours, hierarchy = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

        approx_contours=[]
        for i, cnt in enumerate(contours):
            # 輪郭の周囲に比例する精度で輪郭を近似する
            arclen = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, arclen*0.00001, True)
            approx_contours.append(approx)
            
            
        x_list = []
        y_list = []
        
        
        
        for i in range(0, len(approx_contours)):
            if len(approx_contours[i]) > 0:
                #画像に対して面積が小さいものをはじく
                if cv2.contourArea(approx_contours[i]) < area//100000:
                    #files[0].split('/')[-1]
                    continue

                buf_np = approx_contours[i].flatten() # numpyの多重配列になっているため、一旦展開する。

                for i, elem in enumerate(buf_np):
                    if i%2==0:
                        x_list.append(elem)
                    else:
                        y_list.append(elem)


        # pandasのSeries型へ一旦変換
        x_df = pd.Series(x_list,dtype="float64")
        y_df = pd.Series(y_list,dtype="float64")
        
        files[0].split('/')[-1]
        # pandasのDataFrame型へ結合と共に、列名も加えて変換
        DF = pd.concat((x_df.rename(r'X'), y_df.rename('Y')), axis=1, sort=False)
        DF.to_csv(tar_path+str()+'.csv', encoding="utf-8", index=False)
        
    print("contour comp")
    
    
def contour_parallel(tar_path,files,pad_files,n):
    
    img_original = cv2.imread(pad_files)

    #画像の面積
    h, w = img_original.shape[:2]
    area = h*w

    # グレースケール変換
    gray = cv2.cvtColor(img_original, cv2.COLOR_RGB2GRAY)

    contours, hierarchy = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    approx_contours=[]
    for i, cnt in enumerate(contours):
        # 輪郭の周囲に比例する精度で輪郭を近似する
        arclen = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, arclen*0.00001, True)
        approx_contours.append(approx)
    
    x_list = []
    y_list = []
    
    for i in range(0, len(approx_contours)):
        if len(approx_contours[i]) > 0:
            #画像に対して面積が小さいものをはじく
            if cv2.contourArea(approx_contours[i]) < area//100000:
                #files[0].split('/')[-1]
                continue

            buf_np = approx_contours[i].flatten() # numpyの多重配列になっているため、一旦展開する。

            for i, elem in enumerate(buf_np):
                if i%2==0:
                    x_list.append(elem)
                else:
                    y_list.append(elem)


    # pandasのSeries型へ一旦変換
    x_df = pd.Series(x_list,dtype="float64")
    y_df = pd.Series(y_list,dtype="float64")

    name = files.split('/')[-1].split('.png')[0]
    #print(name)
    
    # pandasのDataFrame型へ結合と共に、列名も加えて変換
    DF = pd.concat((x_df.rename(r'X'), y_df.rename('Y')), axis=1, sort=False)
    DF.to_csv(tar_path+name+'.csv', encoding="utf-8", index=False)
        
    #print("contour comp")
    
def rotate(path, tar_path, rot_path, data_path, name_files):
    
    # csvの読み込み
    files=sorted(glob.glob(tar_path+'*.csv'), key=numericalSort)
    
    #csvから読み込んでnp.arrayに変換
    data = pd.read_csv(files[0]).values.tolist()
    _data = np.array(data)

    x = _data[:,0]
    y = _data[:,1]

    #重心
    #xg1,yg1 = sum(x)//len(x), sum(y)//len(y)
    #中心
    xc,yc = (max(x)+min(x))//2, (max(y)+min(y))//2
    
    num = 0
    for n in trange(len(files)):
        #print(files[n])
        #csvから読み込んでnp.arrayに変換
        data = pd.read_csv(files[n]).values.tolist()
        _data = np.array(data)
        
        x1 = []
        y1 = []
        
        x1=_data[:,0]
        y1=_data[:,1]
        
        
        #重心
        #xg1,yg1 = sum(x1)//len(x1), sum(y1)//len(y1)
        #中心
        xg1,yg1 = (max(x1)+min(x1))//2, (max(y1)+min(y1))//2
        
        
        x_list = []
        y_list = []
        z_list = []
        rotate_x = []
        rotate_y = []
        name = []
        padding_x = []
        padding_y = []
        
        
        # numpyの多重配列になっているため、一旦展開する。    
        buf_np = _data.flatten()
        
        
        #z軸
        num = files[n].split('/')[-1].split('.png')[0].split('_')[2]
        
        for i, elem in enumerate(buf_np):
            if i%2==0:
                x_list.append(elem + (xc-xg1))
                rotate_x.append(xc-xg1)
            
            else:
                y_list.append(elem + (yc-yg1))
                rotate_y.append(yc-yg1)
                z_list.append(int(num))
            
        
        name = [name_files[n].split('/')[-1]] * len(x_list)
        img_name = pd.Series(name,dtype="string")
        
        padding = np.loadtxt(path + "size.txt")
        padding_x = [padding[0]] * len(x_list)
        padding_y = [padding[1]] * len(x_list)
        
        padding_x_df = pd.Series(padding_x,dtype="float64")
        padding_y_df = pd.Series(padding_y,dtype="float64")
                   
        # pandasのSeries型へ一旦変換
        x_df = pd.Series(x_list,dtype="float64")
        y_df = pd.Series(y_list,dtype="float64")
        z_df = pd.Series(z_list,dtype="float64")
        
        rotate_x_df = pd.Series(rotate_x,dtype="float64")
        rotate_y_df = pd.Series(rotate_y,dtype="float64")
        
        # pandasのDataFrame型へ結合と共に、列名も加えて変換
        DF = pd.concat((x_df.rename(r'X'), y_df.rename('Y'), z_df.rename('Z')), axis=1, sort=False)
        DF.to_csv(rot_path+str(n)+'.csv', encoding="utf-8", index=False)
        
        # 画像名, x,y,z, パディング後のwidth, height, 中心にずらした移動量x,y
        DF = pd.concat((img_name.rename(r'Name'), x_df.rename('X'), y_df.rename('Y'), z_df.rename('Z'), 
                        padding_x_df.rename('Padding_x'), padding_y_df.rename('Padding_y'), rotate_x_df.rename('Center_x'), rotate_y_df.rename('Center_y')), axis=1, sort=False)
        DF.to_csv(data_path+str(n)+'.csv', encoding="utf-8", index=False)
        
        #DF = pd.concat((rotate_x_df.rename(r'Center_x'), rotate_y_df.rename('Center_y')), axis=1, sort=False)
        #DF.to_csv(cen_path+str(n)+'_Center.csv', encoding="utf-8", index=False)
        
    print("rotate comp")

    
def rotate_parallel(tar_files, rot_path, data_path, xy, files, n):
    
    #csvから読み込んでnp.arrayに変換
    data = pd.read_csv(tar_files).values.tolist()
    _data = np.array(data)

    x1 = []
    y1 = []

    x1=_data[:,0]
    y1=_data[:,1]
    
    #重心
    #xg1,yg1 = sum(x1)//len(x1), sum(y1)//len(y1)
    #中心
    xg1,yg1 = (max(x1)+min(x1))//2, (max(y1)+min(y1))//2
    
    x_list = []
    y_list = []
    z_list = []
    rotate_x = []
    rotate_y = []
    name = []
    padding_x = []
    padding_y = []
    
    # numpyの多重配列になっているため、一旦展開する。    
    buf_np = _data.flatten()
    
    #z軸
    #print(files)
    num = files.split('/')[-1].split('.png')[0].split('_')[2]
    
    for i, elem in enumerate(buf_np):
        if i%2==0:
            
            x_list.append(elem + (xy[0]-xg1))
            
            rotate_x.append(xy[0]-xg1)
        else:
            
            y_list.append(elem + (xy[1]-yg1))
            
            rotate_y.append(xy[1]-yg1)
            
            z_list.append(int(num)*20)
    
    
    #名前
    name = [files.split('/')[-1]] * len(x_list)    
    img_name = pd.Series(name,dtype="string")
    
    padding_x = [xy[0]] * len(x_list)
    padding_y = [xy[1]] * len(x_list)
    
    # pandasのSeries型へ一旦変換
    padding_x_df = pd.Series(padding_x,dtype="float64")
    padding_y_df = pd.Series(padding_y,dtype="float64")

    x_df = pd.Series(x_list,dtype="float64")
    y_df = pd.Series(y_list,dtype="float64")
    z_df = pd.Series(z_list,dtype="float64")
    
    rotate_x_df = pd.Series(rotate_x,dtype="float64")
    rotate_y_df = pd.Series(rotate_y,dtype="float64")
    
    
    # pandasのDataFrame型へ結合と共に、列名も加えて変換
    DF = pd.concat((x_df.rename(r'X'), y_df.rename('Y'), z_df.rename('Z')), axis=1, sort=False)
    DF.to_csv(rot_path+files.split('/')[-1].split('.png')[0]+'_rotate.csv', encoding="utf-8", index=False)
    
    # 画像名, x,y,z, パディング後のwidth, height, 中心にずらした移動量x,y
    DF = pd.concat((img_name.rename(r'Name'), x_df.rename('X'), y_df.rename('Y'), z_df.rename('Z'), 
                    padding_x_df.rename('Padding_x'), padding_y_df.rename('Padding_y'), rotate_x_df.rename('Center_x'), rotate_y_df.rename('Center_y')), axis=1, sort=False)
    DF.to_csv(data_path+files.split('/')[-1].split('.png')[0]+'_bd5.csv', encoding="utf-8", index=False)
    
    #print(data_path)
    
    #DF = pd.concat((rotate_x_df.rename(r'Center_x'), rotate_y_df.rename('Center_y')), axis=1, sort=False)
    #DF.to_csv(cen_path+str(n)+'_Center.csv', encoding="utf-8", index=False)
        
    #print("rotate comp")

def all_csv(rot_path,path,i):
    
    All_Files=sorted(glob.glob(rot_path+'*.csv'), key=numericalSort)
    
    # フォルダ中の全csvをマージ
    list = []
    for file in All_Files:
        list.append(pd.read_csv(file))
    df = pd.concat(list, sort=False)
    
    # csv出力
    df.to_csv(path+'datas_'+str(i)+'.csv', encoding='utf_8', index=False)
    
    print("all_csv comp")
    
def tag(files,tag_path,n):
    img = cv2.imread(files[n])
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    dst = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY_INV, 45, np.std(gray))
    
    points = np.argwhere(dst2 > 0)
    
    if label == L:
        dbscan = DBSCAN(eps=4, min_samples=40)
    else:
        dbscan = DBSCAN(eps=5, min_samples=70)
        
    dbscan.fit(points)
    
    # 密度が閾値以上の点だけを取得
    dense_points = points[dbscan.labels_ != -1]
    
    # 結果を画像に描画
    result = np.zeros_like(img)
    for point in dense_points:
        result[point[0], point[1]] = 255
    
    cv2.imwrite(tag_path+str(n)+'.png',result)
    
def img_to_tag(pad_files, tar_files, name_files, tag_csv_path, xy, n):
    
    img = cv2.imread(pad_files[n], cv2.IMREAD_GRAYSCALE)
    
    points = np.column_stack(np.where(img != 255))
    data = points.flatten()
    #x1 = np.array(points[:,0])
    #y1 = np.array(points[:,1])
    
    
    xc = np.array(xy[0])
    yc = np.array(xy[1])
    
    if len(points) < 2:
        return
    
    else:
        x1 = []
        y1 = []
        
        x_list = []
        y_list = []
        z_list = []
        rotate_x = []
        rotate_y = []
        name = []
        padding_x = []
        padding_y = []
        
        tar_data = pd.read_csv(tar_files[n]).values.tolist()
        _data = np.array(tar_data)
        
        x1=_data[:,0]
        y1=_data[:,1]
        
        #中心
        xc1,yc1 = (max(x1)+min(x1))//2, (max(y1)+min(y1))//2
        
        num = name_files[n].split('/')[-1].split('.png')[0].split('_')[2]
        
        for m, elem in enumerate(data):
            if m%2==0:
                x_list.append(elem + (xc-xc1))
                rotate_x.append(xc-xc1)
            else:
                y_list.append(elem + (yc-yc1))
                rotate_y.append(yc-yc1)
                z_list.append(int(num)*20)
                
        padding_x = [xy[0]] * len(x_list)
        padding_y = [xy[1]] * len(x_list)
        
        # pandasのSeries型へ一旦変換
        padding_x_df = pd.Series(padding_x,dtype="float64")
        padding_y_df = pd.Series(padding_y,dtype="float64")
        
        rotate_x_df = pd.Series(rotate_x,dtype="float64")
        rotate_y_df = pd.Series(rotate_y,dtype="float64")
        
        # pandasのSeries型へ一旦変換
        x_df = pd.Series(x_list, dtype="float64")
        y_df = pd.Series(y_list, dtype="float64")
        z_df = pd.Series(z_list, dtype="float64")
        
        name = [name_files[n].split('/')[-1]] * len(x_list)
        img_name = pd.Series(name,dtype="string")
        
        #DF = pd.concat((x_df.rename(r'X'), y_df.rename('Y'), z_df.rename('Z')), axis=1, sort=False)
        DF = pd.concat((img_name.rename(r'Name'), x_df.rename('X'), y_df.rename('Y'), z_df.rename('Z'), 
                        padding_x_df.rename('Padding_x'), padding_y_df.rename('Padding_y'), rotate_x_df.rename('Center_x'), rotate_y_df.rename('Center_y')), axis=1, sort=False)
        #z = np.array(z)
        #xyz = np.concatenate((points, z), axis = 1, dtype='int')
        #xyz = np.concatenate((x_c,y_c, z), axis = 1, dtype='int')
        #DF = pd.DataFrame(xyz,columns =['X','Y','Z'])
        # csv出力
        DF.to_csv(tag_csv_path+str(name_files[n].split('/')[-1])+'.csv', encoding='utf_8', index=False)
    

## Create contour data

In [ ]:
if __name__ == '__main__':
    warnings.filterwarnings('ignore')
    
    # 1 if all imaging datasets are download from the internet,
    # 0 if some imaging datasets have already been download locally, 
    
    DOWNLOAD_ALL = 0
    
    if DOWNLOAD_ALL == 0:
        dataset_start = 67
        dataset_end = 69
    
    elif DOWNLOAD_ALL == 1:
        dataset_start = 0
        dataset_end = 85
    
    for t in trange(0,5):
        print(t)
        for i in trange(dataset_start, dataset_end):
            
            path = 'dataset/'+str(i)+'/'
            
            files=sorted(glob.glob(path+'[D]*/*.png'), key=numericalSort)
            if len(files) == 0:
                files=sorted(glob.glob(path+'[G]*/*.png'), key=numericalSort)
            print(len(files))
            
            version = '_v1'
            
            #The number of cores of the cpu to use
            process = 6
            #process = 1
            
            com_path = path + 'compare'+version+'/'
            mkdir(com_path)
            
            th_path = path + 'k-means_threshold'+version+'/'
            mkdir(th_path)
            
            pad_path = path + 'img_padding'+version+'/'
            mkdir(pad_path)
            
            tar_path = path + 'target_contour'+version+'/'
            mkdir(tar_path)
            
            rot_path = path + 'rotate_filter'+version+'/'
            mkdir(rot_path)
            
            data_path = path + 'ImageData'+version+'/'
            mkdir(data_path)
            
            a = len(files) - len(files) % process
            
            #nomal task
            if t == 0:
                for n in trange(len(files)//process):
                    with concurrent.futures.ProcessPoolExecutor(max_workers=process) as executor:
                        for m in range(process):
                            executor.submit(compare, com_path, files[n*process+m], n*process+m)
                            
                with concurrent.futures.ProcessPoolExecutor(max_workers=process) as executor:
                    for a_i in range(a,len(files)):
                        executor.submit(compare, com_path, files[a_i], a_i)
                        
            #######################################################
            
            if t == 1:
                #com_path = path + 'compare_v1/'
                com=sorted(glob.glob(com_path+'*.txt'), key=numericalSort)
                com_files=[]
                
                for d in com:
                    com_files.append(np.loadtxt(d))
                    
                k_num = kmeans(com_files)
                
                for n in trange(len(files)//process):
                    with concurrent.futures.ProcessPoolExecutor(max_workers=process) as executor:
                        for m in range(process):
                            executor.submit(threshhold, th_path, files[n*process+m], com_files[n*process+m], k_num, n*process+m)
                            
                with concurrent.futures.ProcessPoolExecutor(max_workers=process) as executor:
                    for a_i in range(a,len(files)):
                        executor.submit(threshhold, th_path, files[a_i], com_files[a_i], k_num, a_i)
            
            #######################################################
            
            if t == 2:
                getSize(files,path)
                size_path = path+"/size.txt"
                
                xy = np.loadtxt(size_path)
                
                k_files=sorted(glob.glob(th_path+'*.png'), key=numericalSort)
                
                for n in trange(len(k_files)//process):
                    with concurrent.futures.ProcessPoolExecutor(max_workers=process) as executor:
                        for m in range(process):
                            executor.submit(padding_parallel, pad_path, xy, files[n*process+m], k_files[n*process+m], n*process+m)
                            
                with concurrent.futures.ProcessPoolExecutor(max_workers=process) as executor:
                    for a_i in trange(a,len(files)):
                        executor.submit(padding_parallel, pad_path, xy, files[a_i], k_files[a_i], a_i)
            
            #######################################################
            
            if t == 3:
                
                pad_files=sorted(glob.glob(pad_path+'*.png'), key=numericalSort)
                for n in trange(len(files)//process):
                    with concurrent.futures.ProcessPoolExecutor(max_workers=process) as executor:
                        for m in range(process):
                            executor.submit(contour_parallel, tar_path, files[n*process+m], pad_files[n*process+m], n*process+m)
                            
                with concurrent.futures.ProcessPoolExecutor(max_workers=process) as executor:
                    for a_i in trange(a,len(files)):
                        executor.submit(contour_parallel, tar_path, files[a_i], pad_files[a_i], a_i)
                        
            #######################################################
            
            if t == 4:
                size_path = path+"/size.txt"
                xy = np.loadtxt(size_path)
                
                tar_files=sorted(glob.glob(tar_path+'*.csv'), key=numericalSort)
                
                #csvから読み込んでnp.arrayに変換
                data = pd.read_csv(tar_files[0]).values.tolist()
                _data = np.array(data)
                
                x = _data[:,0]
                y = _data[:,1]
                
                #中心
                xy = [(max(x)+min(x))//2, (max(y)+min(y))//2]
                
                
                for n in trange(len(files)//process):
                    with concurrent.futures.ProcessPoolExecutor(max_workers=process) as executor:
                        for m in range(process):
                            executor.submit(rotate_parallel, tar_files[n*process+m], rot_path, data_path, xy, files[n*process+m], n*process+m)
                            
                with concurrent.futures.ProcessPoolExecutor(max_workers=process) as executor:
                    for a_i in trange(a,len(files)):
                        executor.submit(rotate_parallel, tar_files[a_i], rot_path, data_path, xy, files[a_i], a_i)
                        
            #######################################################

  0%|          | 0/5 [00:00<?, ?it/s]

0


  0%|          | 0/2 [00:00<?, ?it/s]

162


  0%|          | 0/27 [00:00<?, ?it/s]

dataset/67/compare_v1/dataset/67/compare_v1/dataset/67/compare_v1/dataset/67/compare_v1/ dataset/67/compare_v1/  dataset/67/G2AN_TM11.5_G/G2AN_TM11.5_0003_G004_1.pngdataset/67/G2AN_TM11.5_G/G2AN_TM11.5_0013_G004_2.png 1 
 dataset/67/G2AN_TM11.5_G/G2AN_TM11.5_0000_G001_1.png4 0
 
 dataset/67/G2AN_TM11.5_G/G2AN_TM11.5_0016_G007_2.pngdataset/67/G2AN_TM11.5_G/G2AN_TM11.5_0010_G001_2.png5 3
dataset/67/compare_v1/ dataset/67/G2AN_TM11.5_G/G2AN_TM11.5_0006_G007_1.png 2

dataset/67/compare_v1/dataset/67/compare_v1/dataset/67/compare_v1/dataset/67/compare_v1/dataset/67/compare_v1/dataset/67/compare_v1/   dataset/67/G2AN_TM11.5_G/G2AN_TM11.5_0020_G001_3.png  dataset/67/G2AN_TM11.5_G/G2AN_TM11.5_0036_G007_4.pngdataset/67/G2AN_TM11.5_G/G2AN_TM11.5_0030_G001_4.png dataset/67/G2AN_TM11.5_G/G2AN_TM11.5_0023_G004_3.png  11 
7dataset/67/G2AN_TM11.5_G/G2AN_TM11.5_0026_G007_3.png 8
 9

6
dataset/67/G2AN_TM11.5_G/G2AN_TM11.5_0033_G004_4.png 10
dataset/67/compare_v1/dataset/67/compare_v1/dataset/67/compare

## Creation of data for tagging area portion

In [ ]:
if __name__ == '__main__':
    for t in trange(3):
        for i in trange(1,85):

            path = str(i)+'/'

            files=sorted(glob.glob(path+'[D]*/*.png'), key=numericalSort)

            if len(files) == 0:
                files=sorted(glob.glob(path+'[G]*/*.png'), key=numericalSort)
                
            
            tag_path = path + 'tag_img'+version+'/'
            mkdir(tag_path)
            pad_path = path + 'tag_img_padding'+version+'/'
            mkdir(pad_path)
            csv_path = path + 'tag_csv'+version+'/'
            mkdir(csv_path)
            
            
            #######################################################
            
            if t == 0:
                getSize(files,path)
                size_path = path+"/size.txt"

                xy = np.loadtxt(size_path)

                files=sorted(glob.glob(path+'k-means_threshhold/*/[0-9]*.png'), key=numericalSort)

                for n in trange(len(files)//process):
                    with concurrent.futures.ProcessPoolExecutor(max_workers=process) as executor:
                        for m in range(process):
                            executor.submit(tag, files, tag_path, n*process+m)

                with concurrent.futures.ProcessPoolExecutor(max_workers=6) as executor:
                    for a_i in trange(a,len(files)):
                        executor.submit(tag, files, tag_path, a_i)
            
            #######################################################
            
            #######################################################
            
            if t == 1:
                getSize(files,path)
                size_path = path+"/size.txt"

                xy = np.loadtxt(size_path)

                files=sorted(glob.glob(path+'k-means_threshhold/*/[0-9]*.png'), key=numericalSort)

                for n in trange(len(files)//process):
                    with concurrent.futures.ProcessPoolExecutor(max_workers=process) as executor:
                        for m in range(process):
                            executor.submit(padding_parallel, pad_path,xy,files,n*process+m)

                with concurrent.futures.ProcessPoolExecutor(max_workers=6) as executor:
                    for a_i in trange(a,len(files)):
                        executor.submit(padding_parallel, pad_path,xy,files,a_i)
            
            #######################################################
            
            #######################################################
            
            if t == 2:
                tar_path = path + 'target_contour'+version+'/'
                tar_files = sorted(glob.glob(tar_path+'*.csv'), key=numericalSort)
                pad_files=sorted(glob.glob(pad_path+'*.png'), key=numericalSort)
                
                size_path = path+"/size.txt"
                xy = np.loadtxt(size_path)

                for n in trange(len(name_files)//process):
                    with concurrent.futures.ProcessPoolExecutor(max_workers=process) as executor:
                        for m in range(process):
                            executor.submit(img_to_tag, pad_files, tar_files, files, tag_csv_path, xy,  n*process+m)

                with concurrent.futures.ProcessPoolExecutor(max_workers=process) as executor:
                    for a_i in range(a,len(name_files)):
                        executor.submit(img_to_tag, pad_files, tar_files, files, tag_csv_path, xy, a_i)
            
            #######################################################